In [17]:
import torch
import torch.nn as nn
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
import pandas as pd


In [19]:
# Load data
df = pd.read_csv('ny_data.csv')
temps = df[['DATE', 'TMAX', 'TMIN', "PRCP"]].values

/var/folders/b9/fkf035lj1y786gv5xh300_qc0000gn/T/ipykernel_12901/2629307008.py:2: DtypeWarning: Columns (7,11,17,23,31,33,35,37,39,41,43,45,47,49,51,53,55,65,67,69,71,73,75,77,79,81,83,85,87,89) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('ny_data.csv')


In [24]:
temps

array([['1970-01-01', 25, 19, 0.0],
       ['1970-01-02', 30, 18, 0.0],
       ['1970-01-03', 36, 25, 0.04],
       ...,
       ['2024-02-22', 45, 35, 0.04],
       ['2024-02-23', 47, 42, 0.04],
       ['2024-02-24', 44, 28, 0.0]], dtype=object)

In [30]:
dates = [datetime.strptime(row[0], "%Y-%m-%d") for row in temps]
# for row in temps:
#   print(type(row[0]))

In [31]:
# Extract features and target
X = np.array([[row[2], row[1], row[3]] for row in temps], dtype=np.float32)
y = np.array([[row[1]] for row in temps], dtype=np.float32)

In [34]:
y

array([[25.],
       [30.],
       [36.],
       ...,
       [45.],
       [47.],
       [44.]], dtype=float32)

In [35]:
# Normalize features
scaler = MinMaxScaler(feature_range=(-1, 1))
X_normalized = scaler.fit_transform(X)



In [36]:
# Convert data to PyTorch tensors
X_tensor = torch.from_numpy(X_normalized).unsqueeze(1)  # Add a dimension for sequence length
y_tensor = torch.from_numpy(y)

In [37]:
# Define LSTM model
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=1):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

In [38]:
# Define hyperparameters
input_size = 3  # Number of features
hidden_size = 64
num_layers = 2
learning_rate = 0.001
num_epochs = 100


In [39]:
# Initialize model
model = LSTM(input_size, hidden_size, num_layers)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [40]:
# Train the model
for epoch in range(num_epochs):
    outputs = model(X_tensor)
    loss = criterion(outputs, y_tensor)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [10/100], Loss: 4282.7783
Epoch [20/100], Loss: 4273.0952
Epoch [30/100], Loss: 4258.8901
Epoch [40/100], Loss: 4235.2285
Epoch [50/100], Loss: 4194.3789
Epoch [60/100], Loss: 4126.3789
Epoch [70/100], Loss: 4023.7444
Epoch [80/100], Loss: 3888.2266
Epoch [90/100], Loss: 3731.6882
Epoch [100/100], Loss: 3569.4670


In [41]:
# Test the model
with torch.no_grad():
    future_date = datetime.strptime("2024-03-10", "%Y-%m-%d")
    future_input = torch.from_numpy(scaler.transform(np.array([[0.03, 21, 31]], dtype=np.float32))).unsqueeze(1)
    predicted_temp = model(future_input).item()
    print(f'Predicted maximum temperature for {future_date.strftime("%Y-%m-%d")}: {predicted_temp:.2f}')

Predicted maximum temperature for 2024-03-10: -0.53
